In [1]:
from dataclasses import dataclass
import itertools
import re

In [18]:
@dataclass
class Vector:
    x: int = 0
    y: int = 0
    z: int = 0

@dataclass
class Moon:
    pos: Vector
    vel: Vector
    delta_v: Vector
    
    def kin(self):
        value = 0
        for axis in 'xyz':
            value += abs(getattr(self.vel, axis))
        return value
    
    def pot(self):
        value = 0
        for axis in 'xyz':
            value += abs(getattr(self.pos, axis))
        return value

    def energy(self):
        return self.pot() * self.kin()

In [44]:
def apply_gravity(moons):
    for moon in moons:
        moon.delta_v = Vector()

    for m1, m2 in itertools.combinations(moons, 2):
        for axis in 'xyz':
            if getattr(m1.pos, axis) > getattr(m2.pos, axis):
                setattr(m1.vel, axis, getattr(m1.vel, axis) - 1)
                setattr(m2.vel, axis, getattr(m1.vel, axis) + 1)
            elif getattr(m1.pos, axis) < getattr(m2.pos, axis):
                setattr(m1.vel, axis, getattr(m1.vel, axis) + 1)
                setattr(m2.vel, axis, getattr(m1.vel, axis) - 1)

In [45]:
def apply_velocity(moons):
    for moon in moons:
        for axis in 'xyz':
            setattr(moon.pos, axis, getattr(moon.pos, axis) + getattr(moon.vel, axis))

# Part 1

In [50]:
moons = []
with open("day12.input") as file:
    for line in file:
        moon = Moon(pos=Vector(*map(int, re.findall(r"=(-?\d+)", line))),
                    vel=Vector(),
                    delta_v=Vector())
        moons.append(moon)

In [51]:
for i in range(1000):
    apply_gravity(moons)
    apply_velocity(moons)

In [52]:
moons

[Moon(pos=Vector(x=-118, y=80, z=25), vel=Vector(x=16, y=-2, z=-4), delta_v=Vector(x=3, y=-3, z=-3)),
 Moon(pos=Vector(x=52, y=-46, z=7), vel=Vector(x=-5, y=-10, z=9), delta_v=Vector(x=-1, y=1, z=-1)),
 Moon(pos=Vector(x=22, y=2, z=-14), vel=Vector(x=-9, y=10, z=4), delta_v=Vector(x=1, y=-1, z=3)),
 Moon(pos=Vector(x=63, y=-49, z=-23), vel=Vector(x=-2, y=2, z=-9), delta_v=Vector(x=-3, y=3, z=1))]

In [53]:
total_energy = sum([moon.energy() for moon in moons])
total_energy

10055

# Part 2

In [159]:
from copy import copy

In [160]:
have_cycled = [None]*len(moons)

In [161]:
moons = []
with open("day12.input") as file:
    for line in file:
        moon = Moon(pos=Vector(*map(int, re.findall(r"=(-?\d+)", line))),
                    vel=Vector(),
                    delta_v=Vector())
        moons.append(moon)

In [162]:
for moon in moons:
    moon.initial_pos = copy(moon.pos)

In [163]:
steps = 0
while not all(have_cycled):
    if (steps % 50000) == 0:
        print("At step", steps)
    steps += 1
    apply_gravity(moons)
    apply_velocity(moons)
    for i, moon in enumerate(moons):
        if have_cycled[i]:
            continue
        if (moon.pos == moon.initial_pos) and moon.vel == Vector():
            print("Moon {} has cycled: {}".format(i, steps))
            have_cycled[i] = steps


At step 0
At step 50000
At step 100000
At step 150000
At step 200000
At step 250000
At step 300000
At step 350000


KeyboardInterrupt: 

In [137]:
have_cycled

[924, 2772, 2772, 924]

In [ ]:
<x=16, y=-11, z=2>
<x=0, y=-4, z=7>
<x=6, y=4, z=-10>
<x=-3, y=-2, z=-4>

In [152]:
import numpy as np
from sympy import lcm
moons = np.array([[16, -11, 2],[0, -4, 7],[6, 4, -10],[-3, -2, -4]])
velocity = moons*0

def calcVel(positions):
    vel = []
    for moon in positions:
        a = moons*0
        a += moon-moons < 0
        a -= moon-moons > 0
        vel.append (np.sum(a,axis=0).tolist())
    return vel

# part 1
for _ in range(1000):
    velocity += calcVel(moons)
    moons += velocity
print (np.sum( np.sum(np.abs(moons),axis=1)*np.sum(np.abs(velocity),axis=1) ))

# part 2
moons = np.array([[16, -11, 2],[0, -4, 7],[6, 4, 10],[-3, -2, -4]])
velocity = moons*0
uniqueX, uniqueY, uniqueZ = set(), set(), set()
addX, addY, addZ = True, True, True
while addX or addY or addZ:
    mn = [m.tobytes() for m in moons.T]
    x, y, z = [v.tobytes()+m for v,m in zip(velocity.T,mn)]
    velocity += calcVel(moons)
    moons += velocity
    if addX and x not in uniqueX:
        uniqueX.add(x)
    else:
        addX = False
    if addY and y not in uniqueY:
        uniqueY.add(y)
    else:
        addY = False
    if addZ and z not in uniqueZ:
        uniqueZ.add(z)
    else:
        addZ = False
print (lcm([len(uniqueX),len(uniqueY),len(uniqueZ)]))

10055
38777942760


In [167]:
len(uniqueX)

286332

In [165]:
len(uniqueY)

108344

In [171]:
len(uniqueZ)

120

In [172]:
lcm([2, 3])

6

In [168]:
286332*108344*120

3722682504960